In [236]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import pickle
import pipreqs
from sklearn.preprocessing import LabelEncoder

In [237]:
dataset = pd.read_excel('PPG-BP dataset.xlsx') 

In [238]:
dataset.head(3)

,Num.,subject_ID,Sex_M_F,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension,Diabetes,cerebral_infarction,cerebrovascular_disease
0,1,2,Female,45,152,63,161,89,97,27.268006,Stage 2 hypertension,NaN,NaN,NaN
1,2,3,Female,50,157,50,160,93,76,20.284799,Stage 2 hypertension,NaN,NaN,NaN
2,3,6,Female,47,150,47,101,71,79,20.888889,Normal,NaN,NaN,NaN


In [239]:
dataset['Diabetes'].value_counts()

Type 2 Diabetes    37
Diabetes            1
Name: Diabetes, dtype: int64

In [240]:
dataset.loc[dataset['Diabetes'] == "Type 2 Diabetes", 'Diabetes'] = 1
dataset.loc[dataset['Diabetes'] == "Diabetes", 'Diabetes'] = 1

dataset.loc[dataset['Sex_M/F'] == "Male", 'Sex_M/F'] = 1
dataset.loc[dataset['Sex_M/F'] == "Female", 'Sex_M/F'] = 2

KeyError: 'Sex_M/F'

In [241]:
dataset['Diabetes'].value_counts()

1    38
Name: Diabetes, dtype: int64

In [242]:
# dataset = dataset.fillna(0)
dataset = dataset.drop(['Num.', 'subject_ID'], axis=1)

In [243]:
dataset.head(100)

,Sex_M_F,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension,Diabetes,cerebral_infarction,cerebrovascular_disease
0,Female,45,152,63,161,89,97,27.268006,Stage 2 hypertension,NaN,NaN,NaN
1,Female,50,157,50,160,93,76,20.284799,Stage 2 hypertension,NaN,NaN,NaN
2,Female,47,150,47,101,71,79,20.888889,Normal,NaN,NaN,NaN
3,Male,45,172,65,136,93,87,21.971336,Prehypertension,NaN,NaN,NaN
4,Female,46,155,65,123,73,73,27.055151,Prehypertension,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Female,80,156,60,123,56,58,24.654832,Prehypertension,NaN,cerebral infarction,NaN
96,Female,79,148,40,128,64,73,18.261505,Prehypertension,NaN,NaN,insufficiency of cerebral blood supply
97,Female,85,155,60,149,71,67,24.973985,Stage 1 hypertension,NaN,NaN,cerebrovascular disease
98,Female,69,150,61,141,67,61,27.111111,Stage 1 hypertension,NaN,cerebral infarction,NaN


In [244]:
dataset['cerebrovascular disease'].value_counts()

KeyError: 'cerebrovascular disease'

In [245]:
le = LabelEncoder()

dataset['Hypertension']            = le.fit_transform(dataset['Hypertension'])
dataset['cerebral infarction']     = le.fit_transform(dataset['cerebral infarction'])
dataset['cerebrovascular disease'] = le.fit_transform(dataset['cerebrovascular disease'])

# dataset = dataset.fillna(0)
dataset.head(100)

KeyError: 'cerebral infarction'

In [246]:
dataset = dataset.fillna(0)

In [247]:
dataset['Diabetes'].value_counts()

0    181
1     38
Name: Diabetes, dtype: int64

In [248]:
diabetes_dataset.groupby('Diabetes').mean()

/var/folders/t7/l_bg6rl90297v4jg3wgrrjgw0000gn/T/ipykernel_80502/3717294162.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  diabetes_dataset.groupby('Diabetes').mean()


,Num.,subject_ID,Age(year),Height(cm),Weight(kg),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),Heart Rate(b/m),BMI(kg/m^2)
Diabetes,,,,,,,,,
Diabetes,25.0,31.000000,66.000000,150.000000,57.000000,182.00000,102.000000,81.000000,25.333333
Type 2 Diabetes,185.0,237.108108,58.972973,161.216216,63.054054,129.72973,72.648649,72.864865,24.211017


In [249]:
dataset.groupby('Diabetes').mean()

/var/folders/t7/l_bg6rl90297v4jg3wgrrjgw0000gn/T/ipykernel_80502/2677169500.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dataset.groupby('Diabetes').mean()


,Age_year,Height_cm,Weight_kg,Systolic Blood Pressure_mmHg,Diastolic Blood Pressure_mmHg,Heart_Rate_b_m,BMI_kg_m2,Hypertension
Diabetes,,,,,,,,
0,56.751381,161.292818,59.624309,127.281768,71.519337,73.756906,22.869509,0.950276
1,59.157895,160.921053,62.894737,131.105263,73.421053,73.078947,24.240552,1.078947


In [250]:
X = dataset.drop(columns = 'Diabetes', axis=1)
Y = dataset['Diabetes']

In [251]:
scaler = StandardScaler()

In [252]:
scaler.fit(X)

ValueError: could not convert string to float: 'Female'

In [253]:
standardized_data = scaler.transform(X)

ValueError: could not convert string to float: 'Female'

In [254]:
X = standardized_data
Y = dataset['Diabetes']

In [255]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [256]:
classifier = svm.SVC(kernel='linear')

In [257]:
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [258]:
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [259]:
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [265]:
input_data = (2, 45, 152, 63, 161, 90, 96, 25, 3, 1, 2)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

/Users/alexandra/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


AttributeError: 'StandardScaler' object has no attribute 'mean_'

In [266]:
import pickle

In [267]:
filename = "trained_model_ppg.pkl"
pickle.dump(classifier, open(filename, 'wb'))

In [263]:
#loading save model
loaded_model = pickle.load(open('trained_model_ppg.pkl', 'rb'))

In [264]:
input_data = (2, 45, 152, 63, 161, 90, 96, 25, 3, 1, 2)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

/Users/alexandra/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


AttributeError: 'StandardScaler' object has no attribute 'mean_'